In [ ]:
poch_len = 30.0
df["onset_s"] = np.arange(len(df)) * epoch_len

# WAKE EPOCHS
wake = df[df["Sleep Stage"] == "W"]
print(len(wake), wake.head())
wake = df[df["Sleep Stage"] == "W"].copy()
wake_epochs = list(zip(wake["onset_s"].to_numpy(), (wake["onset_s"] + wake["Duration[s]"]).to_numpy()))
t0_wake, t1_wake = wake_epochs[0]
dur_wake = float(t1_wake - t0_wake)
t0_wake, t1_wake, dur_wake

# REM EPOCHS
rem = df[df["Sleep Stage"] == "R"]
print(len(rem), rem.head())

rem = df[df["Sleep Stage"] == "R"].copy()
rem_epochs = list(zip(rem["onset_s"].to_numpy(), (rem["onset_s"] + rem["Duration[s]"]).to_numpy()))

t0_rem, t1_rem = rem_epochs[0]
dur_rem = float(t1_rem - t0_rem)
t0_rem, t1_rem, dur_rem

#S1 EPOCHS
S1 = df[df["Sleep Stage"] == "S1"]
print(len(S1), S1.head())

S1 = df[df["Sleep Stage"] == "S1"].copy()
S1_epochs = list(zip(S1["onset_s"].to_numpy(), (S1["onset_s"] + S1["Duration[s]"]).to_numpy()))

t0_S1, t1_S1 = S1_epochs[0]
dur_S1 = float(t1_S1 - t0_S1)
t0_S1, t1_S1, dur_S1

# S2 EPOCHS
S2 = df[df["Sleep Stage"] == "S2"]
print(len(S2), S2.head())

S2 = df[df["Sleep Stage"] == "S2"].copy()
S2_epochs = list(zip(S2["onset_s"].to_numpy(), (S2["onset_s"] + S2["Duration[s]"]).to_numpy()))
t0_S2, t1_S2 = S2_epochs[0]
dur_S2 = float(t1_S2 - t0_S2)
t0_S2, t1_S2, dur_S2

# S3 EPOCHS
S3 = df[df["Sleep Stage"] == "S3"]
print(len(S3), S3.head())

S3 = df[df["Sleep Stage"] == "S3"].copy()
S3_epochs = list(zip(S3["onset_s"].to_numpy(), (S3["onset_s"] + S3["Duration[s]"]).to_numpy()))
t0_S3, t1_S3 = S3_epochs[0]
dur_S3 = float(t1_S3 - t0_S3)
t0_S3, t1_S3, dur_S3

# S4 EPOCHS
S4 = df[df["Sleep Stage"] == "S4"]
print(len(S4), S4.head())
S4 = df[df["Sleep Stage"] == "S4"].copy()
S4_epochs = list(zip(S4["onset_s"].to_numpy(), (S4["onset_s"] + S4["Duration[s]"]).to_numpy()))
t0_S4, t1_S4 = S4_epochs[0]
dur_S4 = float(t1_S4 - t0_S4)
t0_S4, t1_S4, dur_S4

In [ ]:
X_raw = eeg.get_data()
X_flt = eeg_filtered.get_data()

print("RAW mean (µV):", np.mean(X_raw)*1e6)
print("FILTERED mean (µV):", np.mean(X_flt)*1e6)
print("RAW std (µV):", np.std(X_raw)*1e6)
print("FILTERED std (µV):", np.std(X_flt)*1e6)

#PSD Comparison
psd_raw, freqs = psd_array_welch(X_raw, sfreq=eeg.info['sfreq'], fmin=0.5, fmax=40.0, n_fft=2048, n_overlap=1024, n_per_seg=2048)
psd_flt, _ = psd_array_welch(X_flt, sfreq=eeg_filtered.info['sfreq'], fmin=0.5, fmax=40.0, n_fft=2048, n_overlap=1024, n_per_seg=2048)

# average across channels
m_raw = psd_raw.mean(axis=0)
m_flt = psd_flt.mean(axis=0)
print("m_raw =", m_raw)
print("m_flt =", m_flt)

# variability across channels (optional)
s_raw = psd_raw.std(axis=0)
s_flt = psd_flt.std(axis=0)
print("s_raw =", s_raw)
print("s_flt =", s_flt)

plt.figure(figsize=(10,5))
for i, ch in enumerate(eeg.ch_names):
    plt.semilogy(freqs, psd_raw[i]*1e12, linestyle="--", linewidth=1, alpha=0.8, label=f"{ch} RAW")
    plt.semilogy(freqs, psd_flt[i]*1e12, linestyle="-",  linewidth=1, alpha=0.8, label=f"{ch} FILTERED")
plt.fill_between(freqs, (m_raw-s_raw)*1e12, (m_raw+s_raw)*1e12, alpha=0.15)
plt.fill_between(freqs, (m_flt-s_flt)*1e12, (m_flt+s_flt)*1e12, alpha=0.15)
plt.xlabel("Frequency (Hz)")
plt.ylabel("PSD (µV²/Hz)")
plt.title("EEG PSD Comparison (mean ± std across channels) RAW(--), FILTERED(-)")
plt.xlim(0.5, 40)
plt.legend(bbox_to_anchor=(1.02, 1), loc="upper left", fontsize=8, ncol=1)
plt.xlim(0.5, 40)
plt.show()

In [ ]:
f = raw.info["sfreq"]

bands = { "delta": (0.5, 4), "theta": (4, 8), "alpha": (8, 12), "beta":  (12, 30), "gamma": (30, 40) }
stage_epochs = {"W": wake_epochs, "R": rem_epochs, "S1": S1_epochs, "S2": S2_epochs, "S3": S3_epochs, "S4": S4_epochs,}

def eeg_bandpower_per_epoch(eeg_filtered, epochs, sf, bands, eeg_chs):
    rows = []
    for i, (t0, t1) in enumerate(epochs):
        a, b = int(t0*sf), int(t1*sf)
        X = eeg_filtered.get_data(start=a, stop=b)  # (n_ch, n_times)
        psd, freqs = psd_array_welch(X, sfreq=sf, fmin=0.5, fmax=40, verbose=False)

        row = {"epoch": i, "start_s": float(t0)}
        for name, (f1, f2) in bands.items():
            idx = (freqs >= f1) & (freqs <= f2)
            bp_ch = np.trapezoid(psd[:, idx], freqs[idx], axis=1)  # per channel
            row[f"{name}_power"] = float(bp_ch.mean())       # mean across channels
        rows.append(row)

    return pd.DataFrame(rows)

In [ ]:
dfs = []
for stage, epochs in stage_epochs.items():
    if len(epochs) == 0:
        continue
    tmp = eeg_bandpower_per_epoch(eeg_filtered, epochs)
    tmp["stage"] = stage
    dfs.append(tmp)

eeg_stage_df = pd.concat(dfs, ignore_index=True)

# Print each sleep stage table
for stage in eeg_stage_df["stage"].unique():
    print(f"Bandpower table for stage {stage} (first 5 rows):")
    display(eeg_stage_df[eeg_stage_df["stage"] == stage].head(5))

# Compute mean bandpower per stage
band_cols = [f"{name}_power" for name in bands.keys()]

mean_table = (eeg_stage_df.groupby("stage")[band_cols].mean().sort_index())

# optional: also std, if you want variability
std_table = ( eeg_stage_df.groupby("stage")[band_cols].std().sort_index())

# Full per-epoch table (first rows)
print("Per-epoch bandpower table (first 10 rows):")
display(eeg_stage_df.head(10))

print("\nMean bandpower per stage:")
display(mean_table)

print("\nStd bandpower per stage:")
display(std_table)


In [ ]:

sf = raw.info["sfreq"]
ecg_ch = ecg_chs[0]  

ecg_sig_total = raw.copy().pick([ecg_ch]).load_data().get_data()[0]
ecg_clean_total = nk.ecg_clean(ecg_sig_total, sampling_rate=sf, method="neurokit")

print(len(ecg_sig_total), len(ecg_clean_total))


In [ ]:
# Filter ECG 
ecg_data = raw.copy().pick(ecg_chs).load_data().get_data()[0]
ecg_filtered = nk.ecg_clean(ecg_data, sampling_rate=raw.info["sfreq"], method="neurokit", lowcut=0.5, highcut=45)

# Create a new Raw object for plotting
info = mne.create_info(ch_names=ecg_chs, sfreq=raw.info["sfreq"], ch_types=['ecg'])
ecg_filtered_raw = mne.io.RawArray(ecg_filtered.reshape(1, -1), info)
fig = ecg_filtered_raw.plot(start=0, duration=dur_rem, scalings=dict(eeg=30e-6), title="ECG — FILTERED", show=True)

In [ ]:

sf = raw.info["sfreq"]
ecg_ch = ecg_chs[0]  

ecg_sig_total = raw.copy().pick([ecg_ch]).load_data().get_data()[0]
ecg_clean_total = nk.ecg_clean(ecg_sig_total, sampling_rate=sf, method="neurokit")

print(len(ecg_sig_total), len(ecg_clean_total))


In [ ]:
rows = []
for i, (t0_rem, t1_rem) in enumerate(rem_epochs):
    a, b = int(t0_rem*sf), int(t1_rem*sf)
    x = ecg_1d[a:b]
    try:
        x_clean = nk.ecg_clean(x, sampling_rate=sf, method="neurokit")
        _, info = nk.ecg_peaks(x_clean, sampling_rate=sf)
        n_peaks = len(info["ECG_R_Peaks"])
        hr_mean = 60.0 * n_peaks / (t1_rem - t0_rem)   # bpm over that epoch
        rows.append({"epoch": i, "start_s": float(t0_rem), "hr_mean_bpm": hr_mean, "ok": True})
    except Exception:
        rows.append({"epoch": i, "start_s": float(t0_rem), "hr_mean_bpm": np.nan, "ok": False})

rem_hr = pd.DataFrame(rows)

plt.figure(figsize=(12,4))
plt.scatter(rem_hr.loc[rem_hr.ok, "start_s"], rem_hr.loc[rem_hr.ok, "hr_mean_bpm"], s=18)
plt.xlabel("Time (s)")
plt.ylabel("Mean HR (bpm)")
plt.title("REM — Mean Heart Rate per 30s Epoch (all REM bouts)")
plt.grid(True)
plt.show()

In [ ]:
def concat_clean_stage(df_stage, sf, gap_s=2.0):
    gap = np.full(int(gap_s * sf), np.nan)
    parts = []
    for _, r in df_stage[df_stage["ok"]].iterrows():
        parts.append(r["clean_seg"])
        parts.append(gap)
    return np.concatenate(parts[:-1]) if len(parts) else np.array([])

ecg_clean_rem  = concat_clean_stage(ecg_R, sf)
ecg_clean_s1   = concat_clean_stage(ecg_S1, sf)
ecg_clean_s2   = concat_clean_stage(ecg_S2, sf)
ecg_clean_s3   = concat_clean_stage(ecg_S3, sf)
ecg_clean_s4   = concat_clean_stage(ecg_S4, sf)
ecg_clean_wake = concat_clean_stage(ecg_W, sf)


In [ ]:
# Indivudual Heart Beat - REM Sleep
nk.ecg_segment(ecg_clean_rem, rpeaks=None, sampling_rate=sf, show=True)
plt.gcf().suptitle("ECG — REM Sleep", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Indivudual Heart Beat - S1 Sleep
nk.ecg_segment(ecg_clean_s1, rpeaks=None, sampling_rate=sf, show=True)
plt.gcf().suptitle("ECG — S1 Sleep", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Indivudual Heart Beat - S2 Sleep
nk.ecg_segment(ecg_clean_s2, rpeaks=None, sampling_rate=sf, show=True)
plt.gcf().suptitle("ECG — S2 Sleep", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Indivudual Heart Beat - S3 Sleep
nk.ecg_segment(ecg_clean_s3, rpeaks=None, sampling_rate=sf, show=True)
plt.gcf().suptitle("ECG — S3 Sleep", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Indivudual Heart Beat - S4 Sleep
nk.ecg_segment(ecg_clean_s4, rpeaks=None, sampling_rate=sf, show=True)
plt.gcf().suptitle("ECG — S4 Sleep", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])

# Indivudual Heart Beat - WAKE Sleep
nk.ecg_segment(ecg_clean_wake, rpeaks=None, sampling_rate=sf, show=True)
plt.gcf().suptitle("ECG — Wake", fontsize=14)
plt.tight_layout(rect=[0, 0, 1, 0.96])

In [ ]:
resp = raw.copy().pick(["TORACE"]).load_data().get_data()[0]

def hpc_metric(ecg, resp, epochs, sf):
    rows = []
    fs_cpc = 4  # Resample ECG and Resp to a common frequency for coherence analysis
    for i, (t0, t1) in enumerate(epochs):
        a, b = int(t0*sf), int(t1*sf)
        x_ecg = ecg[a:b]
        x_resp = resp[a:b]
        try:
             _, info = nk.ecg_peaks(x_ecg, sampling_rate=sf)
             rpeaks = info["ECG_R_Peaks"]
             rr = np.diff(rpeaks) / sf  # RR intervals in seconds
             t_rr = np.cumsum(rr)  # Time points of RR intervals
             t_grid = np.arange(t_rr[0], t_rr[-1], 1/fs_cpc)
             rr_resampled = np.interp(t_grid, t_rr, rr)  # Resample RR intervals to common grid
             t_resp_grid = np.arange(len(x_resp))/sf
             x_resp_resampled = np.interp(t_grid, t_resp_grid, x_resp)  # Resample Resp to common grid
             nps = min(64, len(rr_resampled))
             f, Cxy = coherence(rr_resampled, x_resp_resampled, fs=fs_cpc, nperseg=nps)

             lf_mask = (f >= 0.01) & (f <= 0.15)
             hf_mask = (f >= 0.15) & (f <= 0.40)
             HFC = np.trapezoid(Cxy[hf_mask], f[hf_mask])
             LFC = np.trapezoid(Cxy[lf_mask], f[lf_mask])
             LFC_HFC_ratio = LFC / HFC if HFC > 0 else np.nan
             rows.append({"epoch": i, "HFC": HFC, "LFC": LFC, "LFC/HFC": LFC_HFC_ratio, "ok": True})
        except Exception as e:
             rows.append({"epoch": i, "start_s": float(t0), "ok": False, "error": str(e)})
             continue   

    return pd.DataFrame(rows)